# Deep Learning

# Utilizing Apache Spark and PySpark for Deep Learning in Neuroimaging Studies

## Why Spark?

- **Efficient Large Data Handling:**
   - Spark is designed for fast computation and handling big datasets, essential for neuroimaging data.

- **PySpark - Python Integration:**
   - PySpark provides a Python interface to Spark, making it easier to implement deep learning algorithms using Python libraries.

- **Distributed Computing:**
   - Spark's distributed nature allows for parallel processing, speeding up the deep learning tasks.

- **MLlib for Machine Learning:**
   - Spark's MLlib offers machine learning algorithms optimized for big data, which we can leverage for our deep learning models.

## Advantages:

- Spark handles our complex and large participant data efficiently.
- PySpark allows seamless integration with Python's deep learning libraries.
- Distributed processing capability accelerates our deep learning computations.
- Scalability ensures that our system can grow with our data needs.

In summary, Apache Spark and PySpark provide a robust, scalable, and efficient platform for deep learning in our neuroimaging study, enabling us to process large datasets with speed and ease.


## 1 - Setting up Spark cluster

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("PySpark_VSCode").getOrCreate()


In [17]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import models
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [18]:
combined_df = pd.read_csv('/home/skander/combined.csv')

In [19]:

# Set 'participant_id' as index
combined_df_with_index = combined_df.set_index('participant_id')

# Extract features for pattern discovery
features = combined_df_with_index  # All columns except 'participant_id'

# Now features DataFrame is ready for unsupervised learning techniques



## 2 - Neuron layer

#### A - Base model

We use both CPU and GPU acceleration in order to make the learning process quick.

In [23]:
# Adjust the input dimension to match the number of features in the dataset, here we have one feature with all our numbers
input_dim = features.shape[1]  # Number of features

# Increase the size of the encoded representations to allow the model to capture more complex patterns
encoding_dim = 128  # Adjusted to increase model capacity

# Input layer
input_layer = Input(shape=(input_dim,))

# Enhanced Encoded layer - where the data is compressed
encoded = Dense(encoding_dim, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.5)(encoded)
# Additional encoding layers for more complexity
encoded = Dense(encoding_dim // 2, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.4)(encoded)
encoded = Dense(encoding_dim // 4, activation='relu')(encoded)

# Enhanced Decoded layer - reconstruction of the input
decoded = Dense(encoding_dim // 4, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.4)(decoded)
decoded = Dense(encoding_dim // 2, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.5)(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Enhanced Autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model with the Adam optimizer and mean squared error loss
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping and reduce learning rate on plateau for better training control
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min', min_lr=0.0001)

# Split the data into training and validation sets for model training and evaluation
from sklearn.model_selection import train_test_split
X_train, X_val, _, _ = train_test_split(features, features, test_size=0.2, random_state=42)

# Train the enhanced autoencoder
autoencoder.fit(X_train, X_train,  # Input and output are the same for an autoencoder
                epochs=200,          # Number of epochs
                batch_size=256,     # Batch size
                shuffle=True,       # Shuffle the data
                validation_data=(X_val, X_val),  # Validation data
                callbacks=[early_stopping, reduce_lr])  # Callbacks for monitoring

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.2444 - val_loss: 0.1749 - lr: 0.0010
Epoch 2/200
1/1 [==============================] - 0s 23ms/step - loss: 0.2311 - val_loss: 0.1748 - lr: 0.0010
Epoch 3/200
1/1 [==============================] - 0s 19ms/step - loss: 0.2396 - val_loss: 0.1746 - lr: 0.0010
Epoch 4/200
1/1 [==============================] - 0s 19ms/step - loss: 0.2374 - val_loss: 0.1744 - lr: 0.0010
Epoch 5/200
1/1 [==============================] - 0s 18ms/step - loss: 0.2316 - val_loss: 0.1743 - lr: 0.0010
Epoch 6/200
1/1 [==============================] - 0s 18ms/step - loss: 0.2292 - val_loss: 0.1740 - lr: 0.0010
Epoch 7/200
1/1 [==============================] - 0s 20ms/step - loss: 0.2294 - val_loss: 0.1739 - lr: 0.0010
Epoch 8/200
1/1 [==============================] - 0s 21ms/step - loss: 0.2286 - val_loss: 0.1737 - lr: 0.0010
Epoch 9/200
1/1 [==============================] - 0s 20ms/step - loss: 0.2313 - val_loss: 0.1735 - lr: 0.0010
Epo

In [21]:
import tensorflow as tf

# Create some tensors and perform operations
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

# Check which device is being used
print('Executing on device:', c.device)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
Executing on device: /job:localhost/replica:0/task:0/device:GPU:0
